In [2]:
import numpy as np
import pandas as pd
import pickle
from env_action.environment import FJSP_under_uncertainties_Env

directory           = 'SMALL'
planning_horizon    = 480*60
critical_machines   = {5, 6, 7, 8, 9, 10, 11, 12, 13, 21, 22, 26, 27}
ReworkProbability   = 0.03
maxtime             = 10
PopSize             = 100
WeibullDistribution = pd.read_excel('DATA/DataMaster.xlsx', sheet_name='Distribution')
K                   = 30
maxJob              = 1320
maxOpe              = 5760


In [3]:
from stable_baselines3 import DQN

with open('VALIDATION/SMALL/pickle_valid_instances_480.pkl', 'rb') as f:
    valid_instances = pickle.load(f)
with open('VALIDATION/SMALL/pickle_valid_scenarios_480.pkl', 'rb') as f:
    valid_scenarios = pickle.load(f)

results = []
method = 'predictive-reactive DQN'
InstanceList = [f'valid{i+1}' for i in range(12)]
ScenarioList = ['A', 'B', 'C']
reward_ratio = 0.99
valenv = FJSP_under_uncertainties_Env(False, False, valid_instances, valid_scenarios, K, WeibullDistribution, critical_machines, 
                                      ReworkProbability, planning_horizon, PopSize, maxtime, maxJob, maxOpe, reward_ratio)
model_path ='models/DQN_default_2024-06-29_17-05-40/DQN_.zip'

model = DQN.load(model_path, env=valenv)

for instance_id in InstanceList:
    print("-----------", instance_id)
    for scenario_id in ScenarioList:
        print("-----", scenario_id)
        # Reset the environment with the new dataset
        valenv.reset(test=True, 
                  datatest=instance_id, 
                  scenariotest=scenario_id)
        
        obs, info = valenv.reset()
        done = False
        
        while not done:
            action, _states = model.predict(obs)
            obs, reward, done, truncated, info = valenv.step(action)
        
        tardiness = valenv.calc_tardiness()
    
        results.append({
                        'Method'    : method,
                        'InstanceID': instance_id,
                        'ScenarioID': scenario_id,
                        'Tardiness' : tardiness
                        })

df = pd.DataFrame(results)
file_name = f"VALIDATION/default_JA_only.xlsx"
df.to_excel(file_name, index=False)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------- valid1
----- A
-------------------------------------------------
Method selection:                    CDR1
-------------------------------------------------
Method selection:                    LFOH-TS
-------------------------------------------------
Method selection:                    LFOH-TS
-------------------------------------------------
Method selection:                    LFOH-TS
-------------------------------------------------
Method selection:                    LFOH-TS
-------------------------------------------------
Method selection:                    LFOH-TS
-------------------------------------------------
Method selection:                    LFOH
----- B
-------------------------------------------------
Method selection:                    LFOH-GA
-------------------------------------------------
Method selection:                    RCRS
------------------------------------------